#BERT Classifier with no distinction between race.

Check if GPU is available.

In [ ]:
!nvidia-smi

Sun Dec  4 00:30:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Import necessary libraries.

In [ ]:
!pip install datasets
!pip install transformers
import pandas as pd
import numpy as np
import random
from scipy.stats import bernoulli
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from scipy.stats import percentileofscore

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW

from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 32.3 MB/s 
     |████████████████████████████████| 182 kB 73.6 MB/s 
     |████████████████████████████████| 132 kB 74.9 MB/s 
     |████████████████████████████████| 212 kB 81.2 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 18.3 MB/s 
     |████████████████████████████████| 7.6 MB 47.8 MB/s 


Preprocess the data.

In [ ]:
# This cell is responsible for preprocessing the dataset.
race_keywords = ['AFRICAN-AMERICAN',
                'AFRICAN AMERICAN',
                'AFRICAN',
                'BLACK',
                'CREOLE',
                'CAUCASIAN',
                'WHITE']

def remove_keys(s, keywords):
    for key in keywords:
        if key in s:
            s = s.replace(key,'')
    return s

def clean_string(s):
    s = s.replace(',',' ')
    s = s.replace('-',' ')
    s = s.replace('\n',' ')
    return s

# Read data and drop nulls
df_adm = pd.read_csv('/content/drive/MyDrive/464ProjectData/ADMISSIONS.csv')
df_nte = pd.read_csv('/content/drive/MyDrive/464ProjectData/NOTEEVENTS.csv')
df_sev = pd.read_csv('/content/drive/MyDrive/464ProjectData/apsiii-score.csv')

df_nte = df_nte.loc[~df_nte.HADM_ID.isnull()].reset_index(drop=True)
df_nte['HADM_ID'] = df_nte['HADM_ID'].apply(int)

df_adm = df_adm[['SUBJECT_ID','HADM_ID','ETHNICITY','DIAGNOSIS']]

# Filter only to nursing notes
df_nte = df_nte[['SUBJECT_ID','HADM_ID','CHARTDATE','CATEGORY','TEXT']]
df_nte = df_nte.loc[df_nte.CATEGORY.str.contains('Nursing')]
df_nte = df_nte.drop_duplicates().reset_index(drop=True)
        
df_sev = df_sev[['subject_id','hadm_id','apsiii']].drop_duplicates()

df = df_adm.merge(df_nte, on=['SUBJECT_ID','HADM_ID'])
df = df.merge(df_sev, left_on=['SUBJECT_ID','HADM_ID'], right_on=['subject_id','hadm_id'])

# Filter to only white/black patients
df = df.loc[df.ETHNICITY.str.contains('WHITE')|df.ETHNICITY.str.contains('BLACK')]
df['ETHNICITY'] = df['ETHNICITY'].apply(lambda x: 'WHITE' if 'WHITE' in x else 'BLACK')

# Removing race-related keywords from text
df['TEXT'] = df['TEXT'].apply(lambda x: x.upper())
df['TEXT'] = df['TEXT'].apply(lambda x: remove_keys(x, race_keywords))

# Clean punctuations
df['TEXT'] = df['TEXT'].apply(clean_string)

df = df.drop(['subject_id','hadm_id'], axis=1)
df = df.dropna(how='any', axis=0)

df.to_csv('/content/drive/MyDrive/464ProjectData/preprocessed.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Set global variables.

In [ ]:
# Turn on GPU computing.
device = torch.device("cuda")
random.seed(42)

# Global configurations.
BATCH_SIZE = 16
EPOCHS     = 10
TAU        = 0.0
DIAG_PROP  = 0.8

In [ ]:
# Samples the treatment disparity between black and white patients.
def sample_disparity(actual_treatment, race, tau, diag_prop):
    race = race.apply(lambda x: 0 if x=='WHITE' else 1)
    bias = race*actual_treatment*tau
    prob = actual_treatment.apply(lambda x: x*diag_prop) - bias
    return bernoulli.rvs(prob)

Define the classifier.

In [ ]:
# The classifier class.
class Classifier(nn.Module):
    # Note that the output corresponds to treatment prescribed (1) or treatment
    # not prescribed (0).
    def __init__(self, input_dim, output_dim=1):
        super(Classifier, self).__init__()
        
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2)
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Linear(128, 32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )

        self.hidden_layer3 = nn.Sequential(
            nn.Linear(32, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x, labels=None):
        output = self.hidden_layer1(x)
        output = self.hidden_layer2(output)
        output = self.hidden_layer3(output)
        return output

Read in the data.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/464ProjectData/preprocessed.csv', lineterminator='\n')
df = df.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])\
                .groupby(['SUBJECT_ID','HADM_ID'])\
                .head(1)

idx_white = random.sample(list(df.loc[df.ETHNICITY=='WHITE'].index), 
              df.loc[df.ETHNICITY=='BLACK'].shape[0])
df = pd.concat([df.loc[idx_white], 
                df.loc[df.ETHNICITY=='BLACK']]).reset_index(drop=True)

df = df[['SUBJECT_ID','ETHNICITY','DIAGNOSIS','TEXT','apsiii']].reset_index(drop=True)
df['apsiii_norm'] = list(map(lambda x: percentileofscore(df['apsiii'], x, 'mean')/100, 
                            df['apsiii']))
df['actual_treatment'] = bernoulli.rvs(df['apsiii_norm'])
df['given_treatment'] = sample_disparity(df['actual_treatment'], df['ETHNICITY'], TAU, DIAG_PROP)

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, _ = RandomOverSampler(random_state=42).fit_resample(df_train, df_train['given_treatment'])
df_test, _ =  RandomOverSampler(random_state=42).fit_resample(df_test,  df_test['given_treatment'])

dataset_train = Dataset.from_pandas(df_train, split='train')
dataset_test  = Dataset.from_pandas(df_test,  split='test')

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test  = DataLoader(dataset_test,  batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
df_stats = df.groupby(['ETHNICITY','actual_treatment']).aggregate({'SUBJECT_ID':'count',
                                                        'given_treatment':'sum'})
df_stats['prop'] = df_stats['given_treatment']/df_stats['SUBJECT_ID']
print(f"Actual Treatment: {np.mean(df['actual_treatment'])}, Given Treatment: {np.mean(df['given_treatment'])}")
df_stats

Actual Treatment: 0.49483421432003843, Given Treatment: 0.39440172993753003


SUBJECT_ID  given_treatment      prop
ETHNICITY actual_treatment                                       
BLACK     0                       2045                0  0.000000
          1                       2117             1675  0.791214
WHITE     0                       2160                0  0.000000
          1                       2002             1608  0.803197

In [ ]:
def encode(examples, tokenizer):
    inputs = examples['TEXT']  
    labels = nn.functional.one_hot(examples['given_treatment'], 
                                   num_classes=2)
    tokenized_inputs = tokenizer(inputs,
                                 return_tensors='pt',
                                 max_length=512,
                                 truncation=True,
                                 padding=True)
    model_inputs = {}
    model_inputs['input_ids']      = tokenized_inputs['input_ids']
    model_inputs['attention_mask'] = tokenized_inputs['attention_mask']
    return model_inputs, labels.float()

In [ ]:
# Use HuggingFace's BERT as the model's encoder.
# https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel
model_enc = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased",
                                            output_scores=True,
                                            output_hidden_states=True,
                                            model_max_length=512)
# Define the models and optimizer
classifier = Classifier(input_dim=512, output_dim=2).to(device)
classifier_optimizer = AdamW(classifier.parameters(), 
                          lr=5e-3)

# Define loss
criterion = nn.BCELoss()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def train_classifier(batch, labels):
    classifier.train()
    classifier_optimizer.zero_grad()
    
    x = model_enc(**batch)                                       # Get embeddings from encoder
    x = x.last_hidden_state.sum(axis=1).detach()                 # Sum across emb_dim, detach
    x = x[:, :512]

    pred = classifier(x)                                         # Make prediction
    loss = criterion(pred, labels)
    loss.backward()
    classifier_optimizer.step()
    return loss

def eval_classifier(batch, labels):
    classifier.eval()
    with torch.no_grad():
        x = model_enc(**batch)                                       # Get embeddings from encoder
        x = x.last_hidden_state.sum(axis=1).detach()                 # Sum across emb_dim, detach
        x = x[:, :512]

        pred = classifier(x)                                         # Make prediction
        loss = criterion(pred, labels)
        return loss

In [ ]:
def train(ep):
    train_loss, test_loss, steps = 0, 0, 0
    for batch in dataloader_train:
        steps += 1
        for i in range(len(batch['ETHNICITY'])):
          batch['ETHNICITY'][i] = 'BLACK'
        #batch.pop('ETHNICITY', None)
        batch, labels           = encode(batch, tokenizer)           # Tokenize and obtain labels
        batch['input_ids']      = batch['input_ids'].to(device)      # Send to GPU
        batch['attention_mask'] = batch['attention_mask'].to(device)
        labels                  = labels.to(device)
        train_loss += float(train_classifier(batch, labels).detach().cpu())
        # if steps % 500 == 0:
        #     torch.save(classifier.state_dict(), f"results/gan/classifier-{ep}-{steps}-{TAU}-{DIAG_PROP}")

    for batch in dataloader_test:
        # batch.pop('ETHNICITY', None)
        for i in range(len(batch['ETHNICITY'])):
          batch['ETHNICITY'][i] = 'BLACK'
        batch, labels           = encode(batch, tokenizer)           # Tokenize and obtain labels
        batch['input_ids']      = batch['input_ids'].to(device)      # Send to GPU
        batch['attention_mask'] = batch['attention_mask'].to(device)
        labels                  = labels.to(device)
        test_loss += float(eval_classifier(batch, labels).detach().cpu())
    
    train_loss /= len(dataloader_train)
    test_loss  /= len(dataloader_test)
    print(f"Epoch {ep}: {train_loss}, {test_loss}")
    return train_loss, test_loss

Train the classifier.

In [ ]:
best_test = np.inf
for ep in range(EPOCHS):
    train_loss, test_loss = train(ep)
    if test_loss < best_test:
        best_test = test_loss
        torch.save(classifier.state_dict(), f"/content/drive/MyDrive/464ProjectData/classifier-{TAU}-{DIAG_PROP}-final")

Epoch 0: 49.81245192425995, 50.3
Epoch 1: 50.00176454626995, 50.15
Epoch 2: 50.024070129092976, 50.0
Epoch 3: 49.994706383806914, 49.85
Epoch 4: 49.994706383806914, 50.0
Epoch 5: 50.003529092539914, 49.85
Epoch 6: 49.99823546126897, 50.0
Epoch 7: 50.00000000753893, 49.85
Epoch 8: 49.996470922537945, 50.0
Epoch 9: 50.005293623732015, 50.15


Evaluate the classifier.

In [ ]:
PNEUMONIA_KEYS = ['PNEUMONIA','PMEUMONIA','PNEUMOMIA',
                  'PNEUMONI','PNAUMONIA','PNEMONIA',
                  'PNEUMNOIA','PNEUMONIN','PNEUMONNIA']
FEVER_KEYS = ['FEVER','FEER']
print(df.shape)
df_filter = df.loc[df['DIAGNOSIS'].apply(lambda s: any([k in s for k in PNEUMONIA_KEYS]))]
print(df_filter.shape)
# df_filter = df.loc[df['DIAGNOSIS'].apply(lambda s: any([k in s for k in FEVER_KEYS]))]

dataset_filter  = Dataset.from_pandas(df_filter,  split='filter')
dataloader_filter = DataLoader(dataset_filter, batch_size=BATCH_SIZE, shuffle=True)

(8324, 8)
(326, 8)


In [ ]:
race_lst, actual_lst, pred_lst, apsiii_lst = [], [], [], []
for idx, batch in enumerate(dataloader_filter):
    # print(f"Predicting: Batch {idx}")
    
    # Obtain actual treatment and race features
    actual_treatment        = nn.functional.one_hot(batch['actual_treatment'], 
                                                    num_classes=2).float()
    race                    = torch.tensor(list(map(lambda x: 1*(x=='BLACK'), 
                                                    batch['ETHNICITY'])))
    apsiii = batch['apsiii']
    
    batch, labels           = encode(batch, tokenizer)           # Tokenize and obtain labels
    batch['input_ids']      = batch['input_ids'].to(device)      # Send to GPU
    batch['attention_mask'] = batch['attention_mask'].to(device)
    labels                  = labels.to(device)
    
    x = model_enc(**batch)                                       # Get embeddings from encoder
    x = x.last_hidden_state.sum(axis=1).detach()                 # Sum across emb_dim, detach
    x = x[:, :512]
    pred = classifier(x).cpu().detach()                          # Make prediction
    
    race_lst.append(race)
    actual_lst.append(actual_treatment)
    pred_lst.append(pred)
    apsiii_lst.append(apsiii)

race_lst   = torch.concat(race_lst)
actual_lst = torch.concat(actual_lst)
pred_lst   = torch.concat(pred_lst) 
apsiii_lst = torch.concat(apsiii_lst)

print(race_lst.shape, actual_lst.shape, pred_lst.shape, apsiii_lst.shape)

idx_0 = torch.where(race_lst==0)[0]
idx_1 = torch.where(race_lst==1)[0]

print(idx_0, idx_1)

acc_1 = float(torch.mean(1.0*(actual_lst[idx_1].argmax(dim=1)==pred_lst[idx_1].argmax(dim=1))))
acc_0 = float(torch.mean(1.0*(actual_lst[idx_0].argmax(dim=1)==pred_lst[idx_0].argmax(dim=1))))
roc_0 = roc_auc_score(y_true  = actual_lst[idx_0].numpy(),y_score = pred_lst[idx_0].numpy())
roc_1 = roc_auc_score(y_true  = actual_lst[idx_1].numpy(),
              y_score = pred_lst[idx_1].numpy())
print(f"White: {acc_0} (Acc) {roc_0} ROC-AUC, Black: {acc_1} (Acc) {roc_1} ROC-AUC")

torch.Size([326]) torch.Size([326, 2]) torch.Size([326, 2]) torch.Size([326])
tensor([  0,   1,   3,   7,   8,   9,  10,  12,  13,  16,  18,  20,  26,  28,
         29,  30,  31,  32,  38,  39,  44,  46,  48,  51,  52,  53,  62,  63,
         66,  69,  72,  73,  74,  75,  76,  77,  78,  82,  83,  85,  88,  89,
         90,  91,  93,  95,  96,  98, 100, 104, 105, 109, 111, 113, 117, 118,
        120, 123, 127, 128, 129, 130, 131, 132, 133, 137, 138, 140, 141, 142,
        143, 144, 145, 146, 149, 152, 154, 155, 158, 160, 161, 163, 164, 166,
        168, 169, 170, 172, 174, 175, 177, 178, 183, 189, 190, 196, 197, 199,
        201, 203, 207, 208, 209, 211, 212, 214, 215, 216, 217, 218, 219, 222,
        223, 225, 227, 228, 229, 230, 232, 234, 237, 239, 242, 243, 245, 250,
        252, 253, 254, 256, 257, 261, 264, 265, 266, 270, 273, 274, 275, 280,
        281, 288, 291, 292, 293, 294, 295, 302, 303, 306, 307, 310, 313, 314,
        317, 319, 321, 324, 325]) tensor([  2,   4,   5,   6,  1

In [ ]:
df_analysis = pd.DataFrame({'race':race_lst.numpy(),
              'actual':np.argmax(actual_lst.numpy(), axis=1),
              'pred_not_prescribe':pred_lst[:,0].numpy(),
              'pred_prescribe':pred_lst[:,1].numpy(),
              'apsiii':apsiii_lst.numpy()})
df_analysis['prescribe'] = 1*(df_analysis['pred_prescribe'] > df_analysis['pred_not_prescribe'])
df_analysis['correct'] = 1*(df_analysis['prescribe']==df_analysis['actual'])

for i in range(8):
    if i<=5:
        temp = df_analysis.loc[(df_analysis.apsiii >= 20*i)&\
                            (df_analysis.apsiii < 20*(i+1))]\
                                .groupby('race')\
                                .aggregate({'correct':'mean', 'prescribe':'mean', 'apsiii':'count'})
    else:
        temp = df_analysis.loc[(df_analysis.apsiii >= 20*i)]\
                                .groupby('race')\
                                .aggregate({'correct':'mean', 'prescribe':'mean', 'apsiii':'count'})
    print(f"Accuracy (Black): {round(100*temp['correct'][1],3)} ({round(100*temp['prescribe'][1],3)}% prescribed, {temp['apsiii'][1]} patients), \t Accuracy (White): {round(100*temp['correct'][0],3)} ({round(100*temp['prescribe'][0],3)}% prescribed, {temp['apsiii'][0]} patients)")

Accuracy (Black): 66.667 (0.0% prescribed, 3 patients), 	 Accuracy (White): 100.0 (0.0% prescribed, 2 patients)
Accuracy (Black): 59.259 (0.0% prescribed, 54 patients), 	 Accuracy (White): 63.158 (0.0% prescribed, 38 patients)
Accuracy (Black): 32.857 (0.0% prescribed, 70 patients), 	 Accuracy (White): 22.857 (0.0% prescribed, 70 patients)
Accuracy (Black): 8.696 (0.0% prescribed, 23 patients), 	 Accuracy (White): 9.677 (0.0% prescribed, 31 patients)
Accuracy (Black): 23.077 (0.0% prescribed, 13 patients), 	 Accuracy (White): 0.0 (0.0% prescribed, 12 patients)
Accuracy (Black): 0.0 (0.0% prescribed, 1 patients), 	 Accuracy (White): 0.0 (0.0% prescribed, 5 patients)
Accuracy (Black): 0.0 (0.0% prescribed, 3 patients), 	 Accuracy (White): 0.0 (0.0% prescribed, 1 patients)


KeyError: ignored

In [ ]:
df_analysis
